In [1]:
import pandas as pd
from backend.src.utils import MySQLAgent, OracleAgent
# from src.plot_tools import cat_value_count_bar_plot, num_value_count_bar_plot
import json
from datetime import datetime

import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

In [2]:
def read_config(path):
    try:
        with open(path, 'r') as file:
            configs = json.load(file)

        return configs
    except FileNotFoundError:
        print(f"The file {path} was not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {path}.")

In [3]:
configs = read_config(path="./backend/conn/connections.json")
job_configs = configs["CREDITREPORT"]['VM1_mysql_conn_info']
sql_agent = MySQLAgent(job_configs)

In [4]:
query = """
    select * from mops_season_report
    WHERE company_id = '1104' AND season = 1
"""
df_mops_season_raw = sql_agent.read_table(query=query)

In [5]:
partitions = ['company_id', 'period_year' ,'season', 'acct_name']

def clean_mops_season_duplicants(df, partitions):

    df['row_seq'] = df.groupby(partitions).cumcount() + 1

    df_output = df[df['row_seq'] == 2].drop(['row_seq'], axis=1) 

    return df_output

In [6]:
df_mops_season = clean_mops_season_duplicants(df_mops_season_raw, partitions=partitions)

In [7]:
df_mops_season

,report_name,company_id,company_name,period_year,season,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,creation_date,seq
102,ProfitAndLose,1104,環泥,112,1,基本每股盈餘,0.42,0.0,1.06,0.0,2024-04-10 13:46:49,38
104,ProfitAndLose,1104,環泥,112,1,繼續營業單位淨利（淨損）,0.42,0.0,1.06,0.0,2024-04-10 13:46:49,40
105,ProfitAndLose,1104,環泥,112,1,稀釋每股盈餘,0.42,0.0,1.06,0.0,2024-04-10 13:46:49,41
114,CashFlowStatement,1104,環泥,112,1,利息收入,-1051.00,0.0,-194.00,0.0,2024-04-10 13:46:50,9
166,BalanceSheet,1104,環泥,112,1,流動資產,0.00,0.0,0.00,0.0,2024-04-10 14:18:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
327,CashFlowStatement,1104,環泥,112,1,籌資活動之淨現金流入（流出）,-89513.00,0.0,129929.00,0.0,2024-04-10 14:18:17,56
328,CashFlowStatement,1104,環泥,112,1,本期現金及約當現金增加（減少）數,55861.00,0.0,29892.00,0.0,2024-04-10 14:18:17,57
329,CashFlowStatement,1104,環泥,112,1,期初現金及約當現金餘額,784464.00,0.0,292032.00,0.0,2024-04-10 14:18:17,58
330,CashFlowStatement,1104,環泥,112,1,期末現金及約當現金餘額,840325.00,0.0,321924.00,0.0,2024-04-10 14:18:17,59


In [8]:
# separate to three tables
# show company_name and company_id separatedly in the frontend


- separate to three tables
- show company_name and company_id separatedly in the frontend


In [9]:
cashflow = df_mops_season[df_mops_season['report_name'] == 'CashFlowStatement'].drop(['report_name', 'company_id', 'company_name', 'creation_date', 'seq'], axis=1)
balance = df_mops_season[df_mops_season['report_name'] == 'BalanceSheet'].drop(['report_name', 'company_id', 'company_name', 'creation_date', 'seq'], axis=1)
profitlost = df_mops_season[df_mops_season['report_name'] == 'ProfitAndLose'].drop(['report_name', 'company_id', 'company_name', 'creation_date', 'seq'], axis=1)

In [10]:
cashflow.to_dict(orient='records')

[{'period_year': 112,
  'season': 1,
  'acct_name': '利息收入',
  'this_year_amt': -1051.0,
  'this_year_percent': 0.0,
  'last_year_amt': -194.0,
  'last_year_percent': 0.0},
 {'period_year': 112,
  'season': 1,
  'acct_name': '營業活動之現金流量－間接法',
  'this_year_amt': 0.0,
  'this_year_percent': 0.0,
  'last_year_amt': 0.0,
  'last_year_percent': 0.0},
 {'period_year': 112,
  'season': 1,
  'acct_name': '繼續營業單位稅前淨利（淨損）',
  'this_year_amt': 315487.0,
  'this_year_percent': 0.0,
  'last_year_amt': 728756.0,
  'last_year_percent': 0.0},
 {'period_year': 112,
  'season': 1,
  'acct_name': '本期稅前淨利（淨損）',
  'this_year_amt': 315487.0,
  'this_year_percent': 0.0,
  'last_year_amt': 728756.0,
  'last_year_percent': 0.0},
 {'period_year': 112,
  'season': 1,
  'acct_name': '折舊費用',
  'this_year_amt': 44550.0,
  'this_year_percent': 0.0,
  'last_year_amt': 44290.0,
  'last_year_percent': 0.0},
 {'period_year': 112,
  'season': 1,
  'acct_name': '攤銷費用',
  'this_year_amt': 890.0,
  'this_year_percent': 0.0,
 

In [11]:
df_mops_season.to_csv('1104_mops_season_analysis.csv', header=True)

In [12]:
df_mops_season.acct_name.unique()

array(['基本每股盈餘', '繼續營業單位淨利（淨損）', '稀釋每股盈餘', '利息收入', '流動資產', '現金及約當現金',
       '透過損益按公允價值衡量之金融資產－流動', '透過其他綜合損益按公允價值衡量之金融資產－流動',
       '按攤銷後成本衡量之金融資產－流動', '合約資產－流動', '應收票據淨額', '應收帳款淨額', '應收帳款－關係人淨額',
       '其他應收款淨額', '本期所得稅資產', '存貨', '預付款項', '其他流動資產', '流動資產合計', '非流動資產',
       '透過損益按公允價值衡量之金融資產－非流動', '透過其他綜合損益按公允價值衡量之金融資產－非流動',
       '按攤銷後成本衡量之金融資產－非流動', '採用權益法之投資', '不動產、廠房及設備', '使用權資產', '投資性不動產淨額',
       '無形資產', '遞延所得稅資產', '其他非流動資產', '非流動資產合計', '資產總額', '流動負債', '短期借款',
       '應付短期票券', '合約負債－流動', '應付票據', '應付帳款', '應付帳款－關係人', '其他應付款',
       '本期所得稅負債', '租賃負債－流動', '其他流動負債', '流動負債合計', '非流動負債', '遞延所得稅負債',
       '租賃負債－非流動', '其他非流動負債', '非流動負債合計', '負債總額', '歸屬於母公司業主之權益', '股本',
       '普通股股本', '股本合計', '資本公積', '資本公積合計', '保留盈餘', '法定盈餘公積', '特別盈餘公積',
       '未分配盈餘（或待彌補虧損）', '保留盈餘合計', '其他權益', '其他權益合計', '歸屬於母公司業主之權益合計',
       '非控制權益', '權益總額', '負債及權益總計', '預收股款（權益項下）之約當發行股數（單位：股）',
       '母公司暨子公司所持有之母公司庫藏股股數（單位：股）', '營業收入合計', '營業成本合計', '營業毛利（毛損）',
       '營業毛利（毛損）淨額', '營業費用', '推銷費用', '管理費用', '研究發展費

In [13]:
# Example: Manual categorization
categories = {
    'Revenue': ['利息收入', '其他收入'],
    'Expenses': ['營業成本', '營業費用'],
    'Assets': ['流動資產', '非流動資產'],
    'Liabilities': ['流動負債', '非流動負債'],
    'Equity': ['股本', '保留盈餘'],
    'Profitability': ['基本每股盈餘', '稀釋每股盈餘', '繼續營業單位淨利（淨損）']
}

In [14]:
data = df_mops_season

In [15]:
# Assuming the classification based on typical financial statement components, we will classify 'acct_name' into categories.

# Define potential keywords for each category for simple keyword-based categorization
assets_keywords = ["現金", "資產", "存貨", "應收"]
liabilities_keywords = ["負債", "應付"]
equity_keywords = ["股本", "普通股", "優先股", "留存收益"]
income_keywords = ["收入", "銷售", "利潤", "淨利"]
expense_keywords = ["成本", "費用"]
cash_flow_keywords = ["現金流"]

# Function to determine the category based on keywords
def categorize_acct_name(acct_name):
    acct_name = acct_name.lower()
    if any(keyword in acct_name for keyword in assets_keywords):
        return "資產相關項目"
    elif any(keyword in acct_name for keyword in liabilities_keywords):
        return "負債相關項目"
    elif any(keyword in acct_name for keyword in equity_keywords):
        return "股東權益項目"
    elif any(keyword in acct_name for keyword in income_keywords):
        return "收入和費用項目"
    elif any(keyword in acct_name for keyword in expense_keywords):
        return "收入和費用項目"
    elif any(keyword in acct_name for keyword in cash_flow_keywords):
        return "現金流量相關項目"
    else:
        return "其他"

# Apply the categorization function to the 'acct_name' column
data['category'] = data['acct_name'].apply(categorize_acct_name)
data[['acct_name', 'category']].drop_duplicates().sort_values(by='category')  # Show unique categories for review


,acct_name,category
102,基本每股盈餘,其他
229,預收股款（權益項下）之約當發行股數（單位：股）,其他
265,非控制權益（綜合損益）,其他
195,短期借款,其他
264,母公司業主（綜合損益）,其他
...,...,...
293,與營業活動相關之資產之淨變動合計,資產相關項目
301,與營業活動相關之資產及負債之淨變動合計,資產相關項目
177,存貨,資產相關項目
176,本期所得稅資產,資產相關項目


In [16]:
data.to_csv('acct_name_check.csv', encoding='utf8')